In [ ]:
!pip install -U transformers

## Local Inference on GPU
Model page: https://huggingface.co/google/gemma-2b

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/google/gemma-2b)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

The model you are trying to use is gated. Please make sure you have access to it by visiting the model page.To run inference, either set HF_TOKEN in your environment variables/ Secrets or run the following cell to login. 🤗

In [ ]:
from huggingface_hub import login
login()

In [ ]:

from transformers import pipeline

pipe = pipeline("text-generation", model="google/gemma-2b")

In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b")

In [ ]:
import torch
torch.cuda.is_available()



In [ ]:

!pip install -q -U torch transformers accelerate bitsandbytes

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


model_id = "google/gemma-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config={"load_in_4bit": True}
)


def interrogate_model(question, num_samples=5):
    print(f"🕵️ INTERROGATING: '{question}'\n")

    prompt = f"Question: {question}\nAnswer in one short sentence:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    responses = []
    for i in range(num_samples):

        outputs = model.generate(
            **inputs,
            max_new_tokens=50,
            temperature=1.0,
            do_sample=True,
            top_k=50
        )
        ans = tokenizer.decode(outputs[0], skip_special_tokens=True).replace(prompt, "").strip()
        responses.append(ans)
        print(f"  Sample {i+1}: {ans}")

    return responses


trick_question = "Who was the first person to land on Mars?"
samples = interrogate_model(trick_question)


fact_question = "What is the capital of France?"
samples_fact = interrogate_model(fact_question)


In [ ]:

!pip install -q sentence-transformers

from sentence_transformers import CrossEncoder
import numpy as np


print("Loading the NLI Judge (DeBERTa)...")
nli_model = CrossEncoder('cross-encoder/nli-deberta-v3-small')

def calculate_consistency(samples):
    print("\n⚖️ CALCULATING SEMANTIC CONSISTENCY (Contradiction-Based)...")
    reference = samples[0]
    contradiction_scores = []

    print(f"Reference Story: '{reference}'")

    for i in range(1, len(samples)):
        hypothesis = samples[i]


        scores = nli_model.predict([reference, hypothesis])


        probs = np.exp(scores) / np.sum(np.exp(scores))


        contradiction_prob = probs[0]
        contradiction_scores.append(contradiction_prob)


        status = "❌ CONFLICTS" if contradiction_prob > 0.5 else "✅ AGREES"
        print(f"  vs Sample {i+1}: {status} (Contradiction Score: {contradiction_prob:.2f})")


    mean_contradiction = np.mean(contradiction_scores)
    final_truth_score = 1.0 - mean_contradiction

    print(f"\n=> FINAL TRUTH SCORE: {final_truth_score:.2f} / 1.0")
    if final_truth_score > 0.60:
        print("Verdict: 🟢 THE MODEL IS CONFIDENT (Consistent Story)")
    else:
        print("Verdict: 🔴 HIGH UNCERTAINTY (Potential Hallucination/Lying!)")

    return final_truth_score


calculate_consistency(france_answers)


france_answers = [
    "Paris. The capital of France is Paris, which is a major city in the country.",
    "Paris. Paris is the capital of France and is a popular tourist destination.",
    "Paris is the capital of France. The capital of France is Paris, and it is in the north of Europe.",
    "Paris. Paris is the capital of France, a country in Europe.",
    "Paris is the capital of France. Please note that there are several other important cities in France, but Paris is the most important and well-known"
]

calculate_consistency(france_answers)

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM


print("Loading REBEL (Triplet Extractor)...")
tokenizer_rebel = AutoTokenizer.from_pretrained('Babelscape/rebel-large')
model_rebel = AutoModelForSeq2SeqLM.from_pretrained('Babelscape/rebel-large')


triplet_extractor = pipeline(
    'text-generation',
    model=model_rebel,
    tokenizer=tokenizer_rebel
)


def extract_triplets(text):
    print(f"\n🔪 EXTRACTING TRIPLETS FROM: '{text}'")


    output = triplet_extractor(
        text,
        return_tensors=True
    )


    generated_token_ids = output[0]["generated_token_ids"]


    extracted_text = triplet_extractor.tokenizer.batch_decode([generated_token_ids])


    extracted_triplets = triplet_extractor.tokenizer.decode(
        generated_token_ids,
        skip_special_tokens=False
    )

    print(f"  Raw Output: {extracted_triplets}")
    return extracted_triplets


test_sentence = "Paris is the capital of France and is a popular tourist destination."
extract_triplets(test_sentence)

In [ ]:
import requests


def parse_rebel_output(raw_text):
    triplets = []

    clean_text = raw_text.replace('<s>', '').replace('</s>', '').strip()


    chunks = clean_text.split('<triplet>')
    for chunk in chunks:
        if not chunk.strip(): continue
        try:

            head_part, rest = chunk.split('<subj>')
            tail_part, rel_part = rest.split('<obj>')

            triplets.append({
                "Subject": head_part.strip(),
                "Object": tail_part.strip(),
                "Relation": rel_part.strip()
            })
        except ValueError:
            pass

    return triplets


def verify_fact_sparql(subject, object_):
    print(f"\n🏛️ THE JUDGE: Verifying connection between '{subject}' and '{object_}'...")

    url = 'https://query.wikidata.org/sparql'


    headers = {
        'User-Agent': 'ProjectSatya/1.0 (Student_Researcher_Test)',
        'Accept': 'application/sparql-results+json'
    }

    query = f"""
    SELECT ?item ?itemLabel ?valLabel WHERE {{
      ?item rdfs:label "{subject}"@en.
      ?item ?prop ?val.
      ?val rdfs:label "{object_}"@en.
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    LIMIT 5
    """

    try:

        r = requests.get(url, params={'query': query}, headers=headers)


        if r.status_code != 200:
            print(f"  ⚠️ Blocked! HTTP Status: {r.status_code}")
            return 0.0

        data = r.json()
        results = data['results']['bindings']

        if len(results) > 0:
            print("  ✅ VERIFIED: Found a direct relationship in the Knowledge Graph!")
            return 1.0
        else:
            print("  ❌ UNVERIFIED: No direct relationship found. Potential Hallucination.")
            return 0.0
    except Exception as e:
        print(f"  ⚠️ API Error: {e}")
        return 0.0


raw_rebel_output = "<s><triplet> Paris <subj> France <obj> country <triplet> France <subj> Paris <obj> capital</s>"

print("1. Parsing Triplets...")
parsed_facts = parse_rebel_output(raw_rebel_output)
for fact in parsed_facts:
    print(f"  - {fact}")

print("\n2. Querying Knowledge Graph...")

test_subject = parsed_facts[0]['Subject']
test_object = parsed_facts[0]['Object']

truth_score = verify_fact_sparql(test_subject, test_object)

In [ ]:
def project_satya_pipeline(question):
    print(f"\n========================================")
    print(f"🚀 PROJECT SATYA FACT-CHECKING PIPELINE")
    print(f"Query: '{question}'")
    print(f"========================================")


    samples = interrogate_model(question, num_samples=3)
    main_answer = samples[0]


    consistency_score = calculate_consistency(samples)

    if consistency_score < 0.60:
        print("\n❌ PIPELINE HALTED: The LLM is hallucinating (Internal Contradiction).")
        return

    print("\n✅ Internal Consistency Passed. Proceeding to Knowledge Graph Verification...")


    raw_triplets_output = extract_triplets(main_answer)


    if isinstance(raw_triplets_output, list) and 'generated_text' in raw_triplets_output[0]:
        triplet_text = raw_triplets_output[0]['generated_text']
    else:
        triplet_text = str(raw_triplets_output)

    parsed_facts = parse_rebel_output(triplet_text)

    if not parsed_facts:
        print("\n⚠️ No extractable facts found in the answer. Cannot verify against graph.")
        return


    verified_count = 0
    for fact in parsed_facts:
        subj, obj = fact['Subject'], fact['Object']
        score = verify_fact_sparql(subj, obj)
        verified_count += score

    final_kg_score = verified_count / len(parsed_facts) if len(parsed_facts) > 0 else 0.0

    print(f"\n========================================")
    print(f"📊 FINAL REPORT")
    print(f"Answer: {main_answer}")
    print(f"Internal Consistency: {consistency_score*100:.1f}%")
    print(f"Knowledge Graph Verification: {final_kg_score*100:.1f}%")

    if final_kg_score > 0.5:
        print(f"Verdict: 🟢 GROUNDED FACT (Safe to display to user)")
    else:
        print(f"Verdict: 🔴 HALLUCINATION (Failed External Verification)")
    print(f"========================================")


print("\n--- RUNNING PIPELINE ON A KNOWN FACT ---")
project_satya_pipeline("The capital city of Japan is")

print("\n--- RUNNING PIPELINE ON A HALLUCINATION TRAP ---")
project_satya_pipeline("The person who invented the internet in 1842 was")

In [ ]:
import numpy as np


us_law_queries = [
    "Under the US Constitution, what does the Fifth Amendment protect a citizen from?",
    "What is the maximum number of years a US President can serve?",
    "Who has the explicit power to declare war under US Federal Law?"
]


indian_law_queries = [
    "Under the Indian Constitution, what are the exact reasonable restrictions on Article 19?",
    "What is the maximum duration of President's Rule in a state under Article 356?",
    "Under the new Bharatiya Nyaya Sanhita (BNS) of 2024, what is the specific penalty for mob lynching?"
]

def run_geographical_bias_audit(queries, domain_name):
    print(f"\n{'='*50}")
    print(f"🌍 INITIATING AUDIT: {domain_name.upper()}")
    print(f"{'='*50}")

    domain_scores = []

    for query in queries:
        # 1. Ask the model 5 times
        samples = interrogate_model(query, num_samples=5)

        # 2. Check how much it contradicts itself
        score = calculate_consistency(samples)
        domain_scores.append(score)

    # Calculate the average confidence for this domain
    avg_score = np.mean(domain_scores)
    print(f"\n📊 {domain_name} AVERAGE TRUTH SCORE: {avg_score*100:.1f}%")
    return avg_score

# 2. Execute the Experiment
print("Starting the Comparative Experiment...")
us_score = run_geographical_bias_audit(us_law_queries, "US Law")
indian_score = run_geographical_bias_audit(indian_law_queries, "Indian Law")

# 3. The Final Research Conclusion
print(f"\n{'='*50}")
print("🏆 EXPERIMENT RESULTS")
print(f"{'='*50}")
print(f"US Law Average Consistency:     {us_score*100:.1f}%")
print(f"Indian Law Average Consistency: {indian_score*100:.1f}%")

gap = (us_score - indian_score) * 100
print(f"\nTHE KNOWLEDGE GAP: {gap:.1f}%")

if gap > 15.0:
    print("CONCLUSION: Hypothesis PROVEN. The model exhibits significant geographical data bias, resulting in severe hallucinations on Indian Legal queries.")
else:
    print("CONCLUSION: Hypothesis REJECTED. The model handles both domains with similar consistency.")